<a href="https://colab.research.google.com/github/Aromie/Python/blob/master/lesson17_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

imdb, info = tfds.load('imdb_reviews', with_info = True, as_supervised = True)

In [ ]:
print(info)

In [4]:
import numpy as np
train_data , test_data = imdb['train'] , imdb['test']


for text, label in train_data:
  print(text)
  print(label)
  break


tf.Tensor(b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.", shape=(), dtype=string)
tf.Tensor(0, shape=(), dtype=int64)


In [5]:
training_sentence = []
training_labels = []

testing_sentence = []
testing_labels = []


In [6]:
for sentence, label in train_data:
  training_sentence.append(sentence.numpy().decode('utf8'))
  training_labels.append(label.numpy())

for sentence, label in test_data:
  testing_sentence.append(sentence.numpy().decode('utf8'))
  testing_labels.append(label.numpy())



In [8]:
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [9]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
vocab_size = 20000
# oov :out of vocab
tokenizer = Tokenizer(num_words = vocab_size , oov_token = '[oov]')
tokenizer.fit_on_texts(training_sentence)
word_dict = tokenizer.word_index

In [ ]:
sequence = tokenizer.texts_to_sequences(training_sentence)
sequence[:10]

In [ ]:
# padding 길이 맞춰주기
MAXLENGTH = 120
sequence = pad_sequences(sequence, maxlen=MAXLENGTH)
sequence[:10]

In [12]:
test_sequence = tokenizer.texts_to_sequences(testing_sentence)
test_sequence = pad_sequences(test_sequence, maxlen=MAXLENGTH)

In [13]:
idx_dict = dict([(value, key) for key, value in word_dict.items()])

In [ ]:
idx_dict

In [ ]:
def decode_review(text):
  result=[]
  for idx in text:
    word = idx_dict.get(idx, '*')
    result.append(word)
  return ' '.join(result)

print('original :', training_sentence[0])
print('seq', sequence[0])
print('decoding : ', decode_review(sequence[0]))

In [21]:
vocab_size=20000
model = tf.keras.Sequential([tf.keras.layers.Embedding(vocab_size, 100, input_length=MAXLENGTH),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(6,activation = 'relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')])

model.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])
model.summary()
model.fit(sequence,training_labels, epochs=10, validation_data=(test_sequence, testing_labels))

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 120, 100)          2000000   
                                                                 
 flatten (Flatten)           (None, 12000)             0         
                                                                 
 dense (Dense)               (None, 6)                 72006     
                                                                 
 dense_1 (Dense)             (None, 1)                 7         
                                                                 
Total params: 2,072,013
Trainable params: 2,072,013
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
782/782 [==============================] - 30s 37ms/step - loss: 0.4131 - accuracy: 0.7982 - val_loss: 0.3310 - val_accuracy: 0.8556
Epoch 2/10
782/782 [==============

In [ ]:
for idx in range(len(model.layers)):
  model.layers[idx]
  

In [24]:
embedding_layers=model.layers[0]
weights = embedding_layers.get_weights()[0]
print(weights.shape)

(20000, 100)
